In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# List all files in data directort
from glob import glob 
flist_all = sorted(glob('./mosaic_data/aerosol-data/datasets/*.tab'))

In [ ]:
files_to_use = [1, 2, 4, 6, 13, 14, 15]
#files_to_use = [1, 2, 4, 6, 13, 15]

blh=[320,100,410,180,290,920,90]

In [ ]:
flist = [flist_all[i] for i in files_to_use]
flist

In [ ]:
# Boundary layer heights for each plot
blh=[320,100,410,180,290,90]

In [ ]:
# Set variable names
varnamesaero = ["DateTime", "z", "p", "Tc", "RH", "TTechC", "RHTech", "N8", "N12", "N150", "Var11", "Var12", "Var13", "Var14", "Var15", "Var16", "Var17", "Var18", "Var19", "Var20", "Var21", "Var22", "Var23", "Var24", "Var25", "Var26", "Var27"]
varnamesmet = ['DateTime', 'p', 'Tc', 'rh', 'wspd', 'wdir', 'z']

In [ ]:
# Create 10m height bins from 0 to 1200 m
zbin = range(0,1201,10)

In [ ]:
for f in flist:
    print(re.findall('(\d{4}-\d{2}-\d{2}_\d{2})', f))

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, constrained_layout=True, sharey=True, figsize=(8,4))

for f,b in zip(flist, blh):
    print(f)
    # Find corresponding meteorology file
    date = re.findall('(\d{4}-\d{2}-\d{2}_\d{2})', f)[0] # find dddd-mm-yy_hh in filename
    f2 = glob(f'./mosaic_data/met-data/datasets/*{date}*')[0]
    
    # Open data 
    df = pd.read_csv(f, delimiter='\t', skiprows=50, names=varnamesaero)
    df2 = pd.read_csv(f2, delimiter='\t', skiprows=29, names=varnamesmet)

    # Trim meterology data to be during the same time range
    t1, t2 = df.DateTime.iloc[0], df.DateTime.iloc[-1]
    df2.loc[(df2['DateTime'] >= t1) & (df['DateTime'] <= t2)]

    # Bin by height
    df['bin'] = np.digitize(df.z, zbin)
    df2['bin'] = np.digitize(df2.z, zbin)

    # Calculate theta
    df2['theta'] = (df2.Tc + 273.15) * (1000/df2.p)**(287/1004)

    # Groupby bin number and take mean of needed variable
    metbin = df2.groupby('bin').mean('theta')
    numbin = df.groupby('bin').mean('N12')
    
    # Add height data based on bin
    metbin['z'] = [zbin[i] for i in metbin.index]
    numbin['z'] = [zbin[i] for i in numbin.index]
    
    # Plot profiles
    metbin.plot(x='theta', y='z',ax=ax1, label=date)
    numbin.plot(x='N12', y='z', ax=ax2, legend=None)

    metbin['thetadiff'] = metbin['theta'] - 2*metbin['theta'].shift(1) + metbin['theta'].shift(2)
    # b = metbin[metbin.thetadiff == metbin.thetadiff.max()].z.values
    # Plot BLH dots
    ax1.plot(metbin.set_index('z').loc[b].theta, b,'o', color='k')
    ax2.plot(numbin.set_index('z').loc[b].N12, b,'o', color='k')
    
plt.ylim(0,1200)
ax1.set_xlim(269, 290)
ax1.grid(visible=True)
ax2.grid(visible=True)

ax1.set_title('(a) Potential Temperature')
ax1.set_xlabel('$\\theta$ [K]')
ax1.set_ylabel('Height [m]')

ax2.set_title('(b) N12 Number Concentration')
ax2.set_xlabel('Concentration [# cm$^{-3}$]')

plt.savefig('plots/01.png', dpi=500)
# ax2.set_xscale('log')

In [ ]:
metbin.z

In [ ]:
metbin['thetadiff'] = metbin['theta'] - 2*metbin['theta'].shift(1) + metbin['theta'].shift(2)


In [ ]:
metbin.plot(x='theta', y='z', label=date)
plt.show()

In [ ]:
metbin.plot(x='thetadiff', y='z', label=date)
